# This notebook test full workflow while using MLRun packgers

This notebook will test :

1. Loging object to MLRun such as: `pd.DataFrame, np.ndarray, str and list`.
2. Unpack `pd.DataFrame, np.ndarray, str and list` and compare them to the original values.
3. Logging object to MLRun such as: `pd.DataFrame, np.ndarray, string, list` with non default artifact type and file format.
4. Unpack outputs from step 3 and validate the expcted values are matched to the original 
5. Pack Custom packager that pack a zip file path to a zip file
6. Unpack Custom packager

In [1]:
import mlrun
import pandas as pd
import numpy as np
import os

In [2]:
project = mlrun.get_or_create_project("test-pack",'./context',user_project=True)

> 2023-07-06 14:20:21,232 [info] Created and saved project: {'name': 'test-pack-gilad', 'from_template': None, 'overwrite': False, 'context': './context', 'save': True}
> 2023-07-06 14:20:21,234 [info] Project created successfully: {'project_name': 'test-pack', 'stored_in_db': True}


## 1.  Loging object to MLRun such as: `pd.DataFrame, np.array, string, list`

Expcted logging result:
* `pd.DataFrame` - as dataset artifact type and default parquet file
* `np.nparray` - as artifact artifact type and default npy file
* `str` - as a run result 
* `list` as a run result

In [3]:
%%writefile ./context/generate-outputs.py
import numpy as np
import pandas as pd


def generate_outputs():
    df_true = pd.DataFrame(
        data={
            **{f"column_{i}": np.arange(0, 1000) for i in range(1, 10)},
        },
    ).set_index(keys=["column_7", "column_8", "column_9"])

    np_example = np.array(
        [[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[1, 2, 3], [4, 5, 6], [7, 8, 9]]]
    )

    example_string = "Example_String"

    example_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, "A"]
    return df_true, np_example, example_string, example_list


Writing ./context/generate-outputs.py


In [4]:
project.set_function("./generate-outputs.py",name='generate-outputs',kind='job',image='mlrun/mlrun',handler='generate_outputs')

> 2023-07-06 14:20:21,260 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}


In [5]:
outputs_run = project.run_function("generate-outputs",returns=["df","np_example","example_string","example_list"])

> 2023-07-06 14:20:21,318 [info] Storing function: {'name': 'generate-outputs-generate-outputs', 'uid': 'df8addace93b4898b507184fbf84ab03', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:20:21,544 [info] Job is running in the background, pod: generate-outputs-generate-outputs-kksch
> 2023-07-06 14:20:27,081 [info] Run execution finished: {'status': 'completed', 'name': 'generate-outputs-generate-outputs'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...bf84ab03,0,Jul 06 14:20:26,completed,generate-outputs-generate-outputs,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=generate-outputs-generate-outputs-kksch,,,"example_string=Example_Stringexample_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 'A']",dfnp_example


> 2023-07-06 14:20:30,088 [info] Run execution finished: {'status': 'completed', 'name': 'generate-outputs-generate-outputs'}


**Validate Run Result**

In [6]:
df_outputs_run = project.get_store_resource(outputs_run.outputs["df"])
assert df_outputs_run.kind == 'dataset'
assert df_outputs_run.target_path.endswith("parquet")

In [7]:
df_outputs_run = project.get_store_resource(outputs_run.outputs["np_example"])
assert df_outputs_run.kind == 'artifact'
assert df_outputs_run.target_path.endswith("npy")

In [8]:
assert outputs_run.outputs["example_string"]=='Example_String'
assert outputs_run.outputs["example_list"]==[1, 2, 3, 4, 5, 6, 7, 8, 9, 'A']

## 2. Unpack `pd.DataFrame, np.ndarray, str and list` and compare them to the original values.

Expected result - 
* run the function without any assertion errors and validate the numpy and pandas are unpacked as expcted 

In [9]:
%%writefile ./context/generate-inputs.py
import numpy as np
import pandas as pd


def generate_inputs(np_example: np.ndarray, df: pd.DataFrame):
    df_true = pd.DataFrame(
        data={
            **{f"column_{i}": np.arange(0, 1000) for i in range(1, 10)},
        },
    ).set_index(keys=["column_7", "column_8", "column_9"])

    # compare dataframe unpack
    assert df_true.compare(df).shape == (0, 0)

    # compare dataframe unpack
    np_true = np.array(
        [[[1, 2, 3], [4, 5, 6], [7, 8, 9]], [[1, 2, 3], [4, 5, 6], [7, 8, 9]]]
    )
    assert np.array_equal(np_true, np_example)

    return "OK"

Writing ./context/generate-inputs.py


In [10]:
project.set_function("./generate-inputs.py",name='generate-inputs',kind='job',image='mlrun/mlrun',handler='generate_inputs')

> 2023-07-06 14:20:30,183 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}


In [11]:
df = outputs_run.outputs["df"]
array_np = outputs_run.outputs["np_example"]


In [12]:
inputs_run = project.run_function("generate-inputs",inputs={"df":df,"np_example":array_np},returns=["state"])

> 2023-07-06 14:20:30,237 [info] Storing function: {'name': 'generate-inputs-generate-inputs', 'uid': 'fdf743c2acc948878e6263ab078457a2', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:20:31,012 [info] Job is running in the background, pod: generate-inputs-generate-inputs-728ss
> 2023-07-06 14:20:36,345 [info] downloading v3io:///projects/test-pack-gilad/artifacts/generate-outputs-generate-outputs/0/np_example.npy to local temp file
> 2023-07-06 14:20:36,494 [info] Run execution finished: {'status': 'completed', 'name': 'generate-inputs-generate-inputs'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...078457a2,0,Jul 06 14:20:35,completed,generate-inputs-generate-inputs,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=generate-inputs-generate-inputs-728ss,dfnp_example,,state=OK,


> 2023-07-06 14:20:40,136 [info] Run execution finished: {'status': 'completed', 'name': 'generate-inputs-generate-inputs'}


**Validate Run Result**

In [13]:
assert inputs_run.outputs["state"] == "OK"

## 3. Logging object to MLRun such as: `pd.DataFrame, np.ndarray, string, list` with non default artifact type and file format.

Excpected output result - 

* df_pkl as an artifact artifact type and pickle file
* df_csv as an artifact artifact type and a csv file
* np_csv as an artifact artifact type and a csv file

In [14]:
%%writefile ./context/generate-outputs-non-default.py
import numpy as np
import pandas as pd
from numpy import asarray


def generate_outputs():
    df_true = pd.DataFrame(
        data={
            **{f"column_{i}": np.arange(0, 1000) for i in range(1, 10)},
        },
    ).set_index(keys=["column_7", "column_8", "column_9"])

    np_example = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])

    df_pkl = df_true
    df_csv = df_true
    np_csv = np_example
    return df_pkl, df_csv, np_csv

Writing ./context/generate-outputs-non-default.py


In [15]:
project.set_function("./generate-outputs-non-default.py",name='generate-outputs-non-default',kind='job',image='mlrun/mlrun',handler='generate_outputs')

> 2023-07-06 14:20:40,168 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}


In [16]:
non_defualt_run = project.run_function('generate-outputs-non-default',returns=["df_pkl:object",
                                                                               {"key":"df_csv","artifact_type":"file","file_format":"csv"},
                                                                               {"key":"np_csv","file_format":"csv"}])

> 2023-07-06 14:20:40,211 [info] Storing function: {'name': 'generate-outputs-non-default-generate-outputs', 'uid': '1f471a847cf14477883c413d6cfddf14', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:20:40,604 [info] Job is running in the background, pod: generate-outputs-non-default-generate-outputs-q4ps5
> 2023-07-06 14:20:46,106 [info] Run execution finished: {'status': 'completed', 'name': 'generate-outputs-non-default-generate-outputs'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...6cfddf14,0,Jul 06 14:20:45,completed,generate-outputs-non-default-generate-outputs,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=generate-outputs-non-default-generate-outputs-q4ps5,,,,df_pkldf_csvnp_csv


> 2023-07-06 14:20:49,726 [info] Run execution finished: {'status': 'completed', 'name': 'generate-outputs-non-default-generate-outputs'}


**Validate Run outputs**

In [17]:
df_pkl_non_defualt_run = project.get_store_resource(non_defualt_run.outputs["df_pkl"])
assert df_pkl_non_defualt_run.kind == 'artifact'
assert df_pkl_non_defualt_run.target_path.endswith("pkl")

In [18]:
df_csv_non_defualt_run = project.get_store_resource(non_defualt_run.outputs["df_csv"])
assert df_csv_non_defualt_run.kind == 'artifact'
assert df_csv_non_defualt_run.target_path.endswith("csv")

In [19]:
np_csv_non_defualt_run = project.get_store_resource(non_defualt_run.outputs["np_csv"])
assert np_csv_non_defualt_run.kind == 'artifact'
assert np_csv_non_defualt_run.target_path.endswith("csv")

## 4. Unpack outputs from step 3 and validate the expcted values are matched to the original 

Excpted result -
* run without assertion errors.

In [20]:
%%writefile ./context/generate-inputs-non-defaults.py
import numpy as np
import pandas as pd
from numpy import asarray


def generate_inputs(df_pkl: pd.DataFrame, df_csv: pd.DataFrame, np_csv: np.ndarray):
    df_true = pd.DataFrame(
        data={
            **{f"column_{i}": np.arange(0, 1000) for i in range(1, 10)},
        },
    ).set_index(keys=["column_7", "column_8", "column_9"])
    # compare dataframe unpack
    assert df_true.compare(df_pkl).shape == (0, 0)
    assert df_true.compare(df_csv).shape == (0, 0)
    # compare dataframe unpack
    np_true = asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

    assert np.array_equal(np_true, np_csv)

    return "OK"


Writing ./context/generate-inputs-non-defaults.py


In [21]:
df_pkl = non_defualt_run.outputs["df_pkl"]
df_csv = non_defualt_run.outputs["df_csv"]
np_csv = non_defualt_run.outputs["np_csv"]

In [22]:
project.set_function("./generate-inputs-non-defaults.py",name='generate-inputs-non-defaults',kind='job',image='mlrun/mlrun',handler='generate_inputs')

> 2023-07-06 14:20:49,871 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}


In [23]:
inputs_run_non_defualt = project.run_function("generate-inputs-non-defaults",inputs={"df_pkl":df_pkl,"df_csv":df_csv,"np_csv":np_csv},returns=["state"])

> 2023-07-06 14:20:49,920 [info] Storing function: {'name': 'generate-inputs-non-defaults-generate-inputs', 'uid': 'a7b7cae614af40f680cd3538d3d62af9', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:20:50,132 [info] Job is running in the background, pod: generate-inputs-non-defaults-generate-inputs-5jgcm
> 2023-07-06 14:20:55,475 [info] downloading v3io:///projects/test-pack-gilad/artifacts/generate-outputs-non-default-generate-outputs/0/df_pkl.pkl to local temp file
> 2023-07-06 14:20:55,507 [info] downloading v3io:///projects/test-pack-gilad/artifacts/generate-outputs-non-default-generate-outputs/0/df_csv.csv to local temp file
> 2023-07-06 14:20:55,542 [info] downloading v3io:///projects/test-pack-gilad/artifacts/generate-outputs-non-default-generate-outputs/0/np_csv.csv to local temp file
> 2023-07-06 14:20:55,652 [info] Run execution finished: {'status': 'completed', 'name': 'generate-inputs-non-defaults-generate-inputs'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...d3d62af9,0,Jul 06 14:20:55,completed,generate-inputs-non-defaults-generate-inputs,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=generate-inputs-non-defaults-generate-inputs-5jgcm,df_pkldf_csvnp_csv,,state=OK,


> 2023-07-06 14:21:00,258 [info] Run execution finished: {'status': 'completed', 'name': 'generate-inputs-non-defaults-generate-inputs'}


## 5. Pack Custom packager that pack a zip file path to a zip file

Expcted result -
* to pack a string as an artifact type artifact and as a zip file

In [24]:
os.makedirs('./custom_pack/')

Writing custome object file

In [25]:
%%writefile ./custom_pack/custom.py
import os
import tempfile
from typing import Tuple

from mlrun import ArtifactType
from mlrun.artifacts import Artifact
from mlrun.datastore import DataItem
from mlrun.errors import MLRunInvalidArgumentError
from mlrun.package import DefaultPackager
from mlrun.package.utils import DEFAULT_ARCHIVE_FORMAT, ArchiveSupportedFormat


class StringPackager(DefaultPackager):
    """
    ``builtins.str`` packager.
    """

    PACKABLE_OBJECT_TYPE = str
    DEFAULT_PACKING_ARTIFACT_TYPE = ArtifactType.FILE
    DEFAULT_UNPACKING_ARTIFACT_TYPE = ArtifactType.PATH

    @classmethod
    def pack_file(
        cls, obj: str, key: str, archive_format: str = DEFAULT_ARCHIVE_FORMAT
    ) -> Tuple[Artifact, dict]:
        """
        Pack a zip value content (pack the file or directory in that path).

        :param obj:            The zip object to pack.
        :param key:            The key to use for the artifact.
        :param archive_format: The archive format to use in case the path is of a directory. Default is zip.

        :return: The packed artifact and instructions.
        """

        # Verify the path is of an existing file:

        # Proceed by path type (file or directory):
        if os.path.isfile(obj):
            # Create the artifact:
            artifact = Artifact(key=key, src_path=os.path.abspath(obj))
            instructions = {"archive_format": "zip", "is_directory": True}
        else:
            raise MLRunInvalidArgumentError(
                f"The given path is not a file nor a directory: '{obj}'"
            )

        return artifact, instructions

    @classmethod
    def unpack_file(
        cls, data_item: DataItem, is_directory: bool = False, archive_format: str = None
    ) -> str:
        """
        Unpack a data item representing a path string. If the path is of a file, the file is downloaded to a local
        temporary directory and its path is returned. If the path is of a directory, the archive is extracted and the
        directory path extracted is returned.

        :param data_item:      The data item to unpack.
        :param is_directory:   Whether the path should be treated as a file or a directory. Files (even archives like
                               zip) won't be extracted.
        :param archive_format: The archive format to use in case the path is of a directory. Default is None - will be
                               read by the archive file extension.

        :return: The unpacked string.
        """
        # Get the file to a local temporary directory:
        path = data_item.local()

        # Mark the downloaded file for future clear:
        cls.add_future_clearing_path(path=path)

        # If it's not a directory, return the file path. Otherwise, it should be extracted according to the archive
        # format:
        if not is_directory:
            return path

        # Get the archive format by the file extension:
        if archive_format is None:
            archive_format = ArchiveSupportedFormat.match_format(path=path)
        if archive_format is None:
            raise MLRunInvalidArgumentError(
                f"Archive format of {data_item.key} ('{''.join(pathlib.Path(path).suffixes)}') is not supported. "
                f"Supported formats are: {' '.join(ArchiveSupportedFormat.get_all_formats())}"
            )

        # Extract the archive:
        archiver = ArchiveSupportedFormat.get_format_handler(fmt=archive_format)
        directory_path = archiver.extract_archive(
            archive_path=path, output_path=os.path.dirname(path)
        )

        # Mark the extracted content for future clear:
        cls.add_future_clearing_path(path=directory_path)

        # Return the extracted directory path:
        return directory_path

Writing ./custom_pack/custom.py


In [26]:
%%writefile ./custom_pack/test-custom.py
import os
import shutil
import time

from custom import StringPackager


def func_custom():
    os.makedirs("./files/")
    with open("./files/test_1.txt", "w") as file:
        file.write("file1")
        file.close()

    with open("./files/test_2.txt", "w") as file:
        file.write("file2")
        file.close()

    with open("./files/test_3.txt", "w") as file:
        file.write("file3")
        file.close()

    shutil.make_archive("test", "zip", "./files/")
    return "./test.zip"

Writing ./custom_pack/test-custom.py


In [27]:
%%writefile ./custom_pack/test-custom-unpack.py
import os
import shutil
import time

from custom import StringPackager

from mlrun import ArtifactType


def func_custom(zip_file: str):
    print(zip_file)
    with open(f"{zip_file}/test_1.txt", "r") as file:
        read = file.read()
        assert read == "file1"
        file.close()

    with open(f"{zip_file}/test_2.txt", "r") as file:
        read = file.read()
        assert read == "file2"
        file.close()

    with open(f"{zip_file}/test_3.txt", "r") as file:
        read = file.read()
        assert read == "file3"
        file.close()

    return "Completed"

Writing ./custom_pack/test-custom-unpack.py


In [28]:
project.add_custom_packager(packager="custom.StringPackager",is_mandatory=True)

In [29]:
project.set_source('v3io:///users/gilad/test_packgers/test.zip',pull_at_runtime=True)

> 2023-07-06 14:21:00,328 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './context', 'error': '/User/full_example/context'}
> 2023-07-06 14:21:00,339 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}
> 2023-07-06 14:21:00,382 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}
> 2023-07-06 14:21:00,395 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}
> 2023-07-06 14:21:00,409 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': 'context', 'error': '/User/full_example/context'}


In [30]:
project.save()

In [31]:
import shutil

In [32]:
shutil.make_archive('test','zip','./custom_pack/')

'/User/full_example/test.zip'

In [33]:
func = project.set_function("./test-custom.py",name="test-custom",kind='job',image='mlrun/mlrun',handler='func_custom',with_repo=True)

In [34]:
test_func = func.run(returns=["test_zip:file"])

> 2023-07-06 14:21:00,744 [info] Storing function: {'name': 'test-custom-func-custom', 'uid': '82330057ed144bf0948c21f1326a3354', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:21:00,962 [info] Job is running in the background, pod: test-custom-func-custom-4rh6n
> 2023-07-06 14:21:05,852 [info] extracting source from v3io:///users/gilad/test_packgers/test.zip to /mlrun/code
> 2023-07-06 14:21:06,317 [info] Run execution finished: {'status': 'completed', 'name': 'test-custom-func-custom'}
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...326a3354,0,Jul 06 14:21:05,completed,test-custom-func-custom,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=test-custom-func-custom-4rh6n,,,,test_zip


> 2023-07-06 14:21:10,101 [info] Run execution finished: {'status': 'completed', 'name': 'test-custom-func-custom'}


## 6. Unpack Custom packager
Expcted result -
* unpack the zip file to the temp folder and check the file content and run without assert issue

In [35]:
func = project.set_function("./test-custom-unpack.py",name="test-custom-unpack",kind='job',image='mlrun/mlrun',handler='func_custom',with_repo=True)

In [36]:
input_zip = test_func.outputs["test_zip"]

In [37]:
func.run(inputs={"zip_file":input_zip},returns=["State"])

> 2023-07-06 14:21:10,151 [info] Storing function: {'name': 'test-custom-unpack-func-custom', 'uid': '2ce99f2705ea4104a9f0c1698be78935', 'db': 'http://mlrun-api:8080'}
> 2023-07-06 14:21:10,365 [info] Job is running in the background, pod: test-custom-unpack-func-custom-b5d8j
> 2023-07-06 14:21:15,260 [info] extracting source from v3io:///users/gilad/test_packgers/test.zip to /mlrun/code
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
test-pack-gilad,...8be78935,0,Jul 06 14:21:15,completed,test-custom-unpack-func-custom,v3io_user=giladkind=jobowner=giladmlrun/client_version=1.4.0-rc18mlrun/client_python_version=3.9.16host=test-custom-unpack-func-custom-b5d8j,zip_file,,,


> 2023-07-06 14:21:19,520 [info] Run execution finished: {'status': 'completed', 'name': 'test-custom-unpack-func-custom'}


In [38]:
db = mlrun.get_run_db()

In [39]:
db.delete_project(name=project.name,deletion_strategy='cascade')

In [40]:
shutil.rmtree('./context/')

In [41]:
shutil.rmtree('./custom_pack/')

In [42]:
os.remove('./test.zip')